# Loading Tables to Snowflake environment
This file serves the purpose of loading datatables to snowflake environment in the event of porting over to new account instance.
The tables in focus will be from the Home Credit Dataset:
- bureau_feature_table
- previous_loan_features
- static_feature_table

Require the following parquet files in the respective locations for the code to work
- data/bureau_feature_table.parquet
- data/previous_loan_features.parquet
- data/static_feature_table.parquet

In [6]:
# All library installations

%pip install feast-azure-provider
%pip install azure-cli
%pip install snowflake-connector-python==2.7.4
%pip install pyarrow==6.0.1

     |████████████████████████████████| 275 kB 35.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.6 MB 49.3 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 11.2 MB/s ta 0:00:01
     |████████████████████████████████| 45 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 26.7 MB 47.8 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 8.4 MB/s  eta 0:00:01
     |████████████████████████████████| 48 kB 7.4 MB/s  eta 0:00:01
     |████████████████████████████████| 445 kB 50.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 58.7 MB/s eta 0:00:01
  Created wheel for pandavro: filename=pandavro-1.5.2-py3-none-any.whl size=2954 sha256=12df6306070879b4a288603c46eab326723944382c113990948a5c092f7b49f0
  Stored in directory: /home/azureuser/.cache/pip/wheels/21/03/ef/f7f69842521378eb3fd932bd3e13341e0b2c1617c56abc4da5
Successfully built pandavro
ERROR: datasets 1.8.0 has requirement pyarrow<4.0.0,>=1.0.0, but you'll have pyarrow

In [3]:
import pandas as pd
import numpy as np
import datetime
from sklearn.base import BaseEstimator, TransformerMixin
# import featuretools as ft
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
# matplotlib and seaborn for plotting
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
import warnings
warnings.filterwarnings("ignore")

In [2]:
from azureml.core import Workspace
from azureml.core import Keyvault
import os

ws = Workspace.from_config()
keyvault = ws.get_default_keyvault()

os.environ["REGISTRY_BLOB_KEY"] = keyvault.get_secret("registrytoken")

UserErrorException: UserErrorException:
	Message: The workspace configuration file config.json, could not be found in /Users/ftt.sakthivel.b/Experiments/feast-snowflake/feature-store/notebooks or its parent directories. Please check whether the workspace configuration file exists, or provide the full path to the configuration file as an argument. You can download a configuration file for your workspace, via http://ml.azure.com and clicking on the name of your workspace in the right top.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "The workspace configuration file config.json, could not be found in /Users/ftt.sakthivel.b/Experiments/feast-snowflake/feature-store/notebooks or its parent directories. Please check whether the workspace configuration file exists, or provide the full path to the configuration file as an argument. You can download a configuration file for your workspace, via http://ml.azure.com and clicking on the name

In [ ]:
#Login Details for snowflake
username = "evan912"
pwd = keyvault.get_secret("evansftestkey")

In [4]:
#Save registry token in environment registry blob key
os.environ["REGISTRY_BLOB_KEY"] = "sp=racwdyti&st=2022-03-16T08:44:21Z&se=2022-12-09T16:44:21Z&spr=https&sv=2020-08-04&sr=b&sig=ZMx%2BpFBaemjiR4j1QRoCGqrbIAQLJ%2F8eJPnx%2Bth4PWQ%3D"

#Snowflake Credentials
username = "evan912"
pwd = 'Evan912912'

In [6]:
import snowflake.connector as snow
from snowflake.connector.pandas_tools import write_pandas

# Snowflake python connector to populate data from datasource to table in snowflake
conn = snow.connect(
   user = username,
   password = pwd,
   account="pn93812.southeast-asia.azure",
   warehouse="COMPUTE_WH",
   database="TEST",
   schema="PUBLIC"
)

#Adds Timezone to all datatime column items in the pandas
def fix_date_cols(df, tz = 'UTC'):
    cols = df.select_dtypes(include=['datetime64[ns]']).columns
    for col in cols:
        df[col] = df[col].dt.tz_localize(tz)

# Wrapper function to upload tables to snowflake
#     data: Pandas Datatable sent to Snowflake
#     table_title: Snowflake tablename
def process_send(data, table_title):
    
    # remove special characters in columns name
    data.columns = data.columns.str.replace("[' ', -]", '_')
    data.columns = data.columns.str.replace("[(, )]", '')
    
    # Add timezone to any datetime type data
    fix_date_cols(data)
    
    #Send data to Snowflake
    try:
        success, nchunks, nrows, _ = write_pandas(conn, data, table_title, quote_identifiers=False, auto_create_table=True)
        print ("Success: " + str(success) + ', Chunks: '+ str (nchunks) + ', Rows uploaded: ' + str (nrows))
    except Exception as e:
        print(e)
    print("Done.")


In [14]:
bureau = pd.read_parquet("data/bureau_feature_table.parquet")
prev = pd.read_parquet("data/previous_loan_features.parquet")
stat = pd.read_parquet("data/static_feature_table.parquet")
application_train = pd.read_csv("data/application_train.csv")

In [17]:
bureau['CREATED_TIMESTAMP']=datetime.datetime.now()
prev['CREATED_TIMESTAMP']=datetime.datetime.now()
stat['CREATED_TIMESTAMP']=datetime.datetime.now()

In [18]:
process_send(application_train, "APPLICATION_TRAIN" )

Success: True, Chunks: 1, Rows uploaded: 307511
Done.


In [19]:
process_send(bureau, "bureau_feature_table" )

Success: True, Chunks: 1, Rows uploaded: 305811
Done.


In [20]:
process_send(prev, "PREVIOUS_LOAN_FEATURES_TABLE" )

Success: True, Chunks: 1, Rows uploaded: 339587
Done.


In [21]:
process_send(stat, "static_feature_table" )

Success: True, Chunks: 1, Rows uploaded: 307511
Done.
